## LangChain: Chains
It is one of the most important key building block of LangChain, namely the `Chain`. The chain usually combines an LLM (Large Language Model) together with a prompt and with this building block, and you can put bunch of these building blocks together to carry out a sequence of operations on your text or your other data. 

Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components.

LangChain provides the Chain interface for such "chained" applications. We define a Chain very generically as a sequence of calls to components, which can include other chains.

### Outline
- LLMChain
- Sequential Chains
  - SimpleSequentialChain
  - SequentialChain
- Router Chain

In [2]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_KEY = os.getenv('OPENAI_KEY')
openai.api_key = OPENAI_KEY

In [3]:
import pandas as pd
df = pd.read_csv("Data.csv")
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


In [7]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0.9, model="gpt-3.5-turbo", openai_api_key=OPENAI_KEY)

#### LLMChain
A LLMChain is the most common type of chain. It consists of a PromptTemplate, a model (either an LLM or a ChatModel), and an optional output parser. This chain takes multiple input variables, uses the PromptTemplate to format them into a prompt. It then passes that to the model. Finally, it uses the OutputParser (if provided) to parse the output of the LLM into a final format.

In [8]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [9]:
chain = LLMChain(llm=llm, prompt=prompt)

product = "Qeen Size Sheet Set"
chain.run(product)

'"Royal Comfort Linens"'

#### Sequential Chains
The next step after calling a language model is to make a series of calls to a language model. This is particularly useful when you want to take the output from one call and use it as the input to another.

It is another type of chains. The idea is to combine multiple chains where the output of the once chain is the input of the next chain.
There is two types of sequential chains:
1. SimpleSequentialChain: Single input/output
2. SequentialChain: multiple inputs/outputs

#### 1. SimpleSequentialChain

In [10]:
from langchain.chains import SimpleSequentialChain

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [11]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [12]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [14]:
product = "Queen Size Sheet Set"
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Royal Comfort Linens
Royal Comfort Linens is a high-quality bedding and linen company that offers luxurious and comfortable products for a peaceful sleep.

> Finished chain.


'Royal Comfort Linens is a high-quality bedding and linen company that offers luxurious and comfortable products for a peaceful sleep.'

#### 2. SequentialChain

In [17]:
from langchain.chains import SequentialChain

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )

second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [18]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [20]:
review = df.Review[5]
review

"Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"

In [21]:
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer is disappointed with the taste and foam quality of the product, suspecting that it may be an old batch or counterfeit compared to the ones bought in stores.',
 'followup_message': "Cher(e) client(e),\n\nNous vous remercions sincèrement d'avoir partagé votre expérience avec notre produit. Nous sommes vraiment désolés d'apprendre que vous avez été déçu(e) par le goût et la qualité de la mousse de notre produit.\n\nNous tenons à vous assurer que nous prenons très au sérieux vos préoccupations et que nous souhaitons résoudre ce problème rapidement. Nous enquêtons actuellement sur la situation afin de déterminer si cela 